In [1]:
from google.cloud import bigquery
from google.cloud.bigquery import job
import pandas as pd
import os
from tqdm import tqdm
import pandas as pd
PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
import statsmodels.api as sm 
import numpy as np

In [5]:
query = """ 
select *
from
  (select *
  from
    (SELECT User_ID, Event_Category, Event_Date, Event_Datetime, Event_Label
    FROM `ballosodeuk.airbridge_warehouse.user_event_log`, unnest(event_detail)
    WHERE 
      Event_Date between "2024-10-01" and "2024-10-15"
      and Event_Category in (
        'tap_go__coupon (App)','tap_go__cp_detail (App)', 'tap_go__cp_detail_buy (App)',
        'view_get__pop (App)','Order Complete (App)')
    ))
where (Event_Category = 'Order Complete (App)' and Event_Label = '쇼핑') or
      (Event_Category = 'tap_go__coupon (App)') or
      (Event_Category = 'tap_go__cp_detail (App)' and Event_Label = '구매_인기쿠폰') or
      (Event_Category = 'tap_go__cp_detail_buy (App)' and Event_Label = '구매하러가기') or
      (Event_Category = 'view_get__pop (App)' and Event_Label in ('상품_구매가능', '상품_구매불가', '상품_구매완료'))
"""

In [6]:
client = bigquery.Client()
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)

In [7]:
df = query_job.to_dataframe()
df


,User_ID,Event_Category,Event_Date,Event_Datetime,Event_Label
0,ddcd4cac-72eb-4f9c-8182-f8fe020afa80,tap_go__coupon (App),2024-10-01,2024-10-01T10:42:29+09:00,구매
1,e14ff5e4-f3b0-4a5a-9ac3-90eeac472b78,tap_go__coupon (App),2024-10-01,2024-10-01T17:32:44+09:00,구매
2,fea30eb3-811c-42fc-b403-7c45375ffc1c,tap_go__coupon (App),2024-10-01,2024-10-01T17:34:10+09:00,구매
3,e5ffc05a-97bf-4da1-9c1f-3a56bd4a676e,tap_go__coupon (App),2024-10-01,2024-10-01T08:15:56+09:00,구매
4,f78e954b-b95f-4c02-9b90-f91fab52322d,tap_go__coupon (App),2024-10-01,2024-10-01T21:52:37+09:00,구매
...,...,...,...,...,...
623993,a283ef9d-3b43-4864-a9be-f4003647e661,view_get__pop (App),2024-10-07,2024-10-07T20:59:49+09:00,상품_구매불가
623994,18f65174-f7f8-4b28-b236-9ed6ba79d698,view_get__pop (App),2024-10-07,2024-10-07T14:11:24+09:00,상품_구매불가
623995,4aef65d9-6298-4f92-8588-47e06329e756,view_get__pop (App),2024-10-07,2024-10-07T21:22:32+09:00,상품_구매불가
623996,b55b0cde-b0b1-4e84-9437-b9cb8bf8045f,view_get__pop (App),2024-10-07,2024-10-07T21:30:22+09:00,상품_구매불가


In [8]:
df.to_csv('./balso_sample.csv')
